In [8]:
import os

from language_table.environments import blocks
from language_table.environments import language_table
from language_table.environments.rewards import block2block

from matplotlib import pyplot as plt
import mediapy
import numpy as np
import tensorflow_datasets as tfds
from icecream import ic

np.set_printoptions(precision=3)

In [9]:
# tf dataset handling
import tensorflow as tf

dataset = tf.data.Dataset.range(10)

for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


2024-06-03 20:57:21.880727: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
DATASET_PATH = "/data/language_table_sim"
builder = tfds.builder_from_directory(DATASET_PATH)
episode_ds = builder.as_dataset(split="train")
print(episode_ds.element_spec)

{'episode_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'steps': DatasetSpec({'action': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'observation': {'effector_target_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'effector_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'instruction': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'rgb': TensorSpec(shape=(360, 640, 3), dtype=tf.uint8, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorShape([]))}


In [11]:
def decode_inst(inst):
    """Utility to decode encoded language instruction"""
    return bytes(inst[np.where(inst != 0)].tolist()).decode("utf-8")

In [12]:
action_mins = []
action_maxs = []

def show_episode(episode):
    """Utility to visualize an episode"""
    print("Episode id:", episode["episode_id"].numpy())
    frames = []
    actions = []
    for step in episode["steps"].as_numpy_iterator():
        frames.append(step["observation"]["rgb"])
        actions.append(step["action"])

    print("step numbers: ", len(frames))
    mediapy.show_video(
        frames, title=decode_inst(step["observation"]["instruction"]), fps=5
    )
    # draw the action sequence
    actions = np.array(actions).T
    print("action sequence:")
    print(actions)

In [13]:
episodes_iter = iter(episode_ds.take(50))

# sample_episodes = tf.data.Dataset.sample_from_datasets(episode_ds)
# episodes_iter = iter(sample_episodes)

for episode in episodes_iter:
    show_episode(episode)

Episode id: b'EiIKGHdvcmtlcl8wODlfZXBfOF8wNl8wNl8yMhAvGDggBDAAKiBmNTI1ODUyODdhMTU3NmI4NWRmYmMyOTliNDk4MTFmYw=='
step numbers:  10


2024-06-03 20:57:24.197835: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


surround the red pentagon with the others This browser does not support the video tag.


action sequence:
[[-3.057e-04 -7.094e-03 -1.165e-02 -1.404e-02 -7.023e-03 -4.685e-03
   8.861e-05 -2.658e-04 -3.987e-04  0.000e+00]
 [ 1.751e-02  9.966e-03  2.447e-03 -7.375e-05 -2.571e-03 -2.567e-03
  -5.076e-03  1.523e-02  2.284e-02  0.000e+00]]
Episode id: b'EiMKGXdvcmtlcl8xMDZfZXBfMTBfMDZfMDZfMjIQORhrIA0wASogZjUyNTg1Mjg3YTE1NzZiODVkZmJjMjk5YjQ5ODExZmM='
step numbers:  51


2024-06-03 20:57:24.591264: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

action sequence:
[[ 3.365e-10 -6.094e-03 -9.479e-03 -1.286e-02 -2.031e-03 -7.448e-03
  -1.354e-02 -1.083e-02 -6.771e-04  8.125e-03  1.269e-02  1.102e-02
   1.083e-02  1.083e-02  1.219e-02  0.000e+00]
 [ 2.031e-03  8.802e-03  7.448e-03  7.448e-03  6.771e-04  2.708e-03
   4.740e-03  4.062e-03 -3.763e-09 -9.479e-03 -2.719e-02 -2.790e-02
  -2.210e-02 -1.016e-02 -8.125e-03  0.000e+00]]
Episode id: b'EiIKGHdvcmtlcl8wNTlfZXBfOF8wNl8wNl8yMhBOGHMgAjAJKiBmNTI1ODUyODdhMTU3NmI4NWRmYmMyOTliNDk4MTFmYw=='
step numbers:  38


2024-06-03 20:57:25.882884: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


"put the blue and green blocks together in the center right, then put the red and yellow blocks together in the bottom center. This browser does not support the video tag."


action sequence:
[[ 1.951e-02  9.037e-03  7.307e-03  3.630e-03  1.471e-03  4.952e-03
   1.632e-02  2.051e-02  7.069e-03 -1.852e-03 -8.029e-03 -6.850e-03
  -6.967e-03 -1.071e-02 -7.315e-03 -4.384e-03 -3.612e-03 -4.348e-03
  -7.979e-03 -6.609e-03 -1.487e-03 -3.002e-03 -6.638e-03 -4.508e-03
  -3.782e-03 -4.533e-03 -1.389e-03 -3.933e-04 -5.750e-04 -1.007e-04
  -1.595e-03 -7.392e-03 -5.551e-03  1.830e-04  3.643e-03  4.251e-03
   8.100e-05  0.000e+00]
 [ 2.545e-03  1.252e-03 -4.359e-03 -2.165e-03 -2.203e-05 -6.368e-03
  -1.126e-02 -9.310e-05  4.052e-03  8.193e-03  1.243e-02  7.017e-03
   5.005e-03  2.373e-03 -1.893e-03  1.170e-04  2.203e-03  2.938e-03
   7.313e-03  5.197e-03  7.658e-04  8.240e-04  7.403e-03  3.058e-03
   2.321e-03  3.807e-03  5.838e-03  1.374e-02  7.259e-03 -3.607e-03
  -1.351e-03  1.569e-02  2.736e-02  6.551e-03  1.716e-02  1.122e-02
   2.900e-03  0.000e+00]]


2024-06-03 20:57:26.040478: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
